### Inicialização do pyspark

In [2]:
import pyspark

spark = pyspark.sql.SparkSession.builder.appName('decision_tree').getOrCreate()
sc = spark.sparkContext

### Implementação de árvore de decisão para classificação MLlib

In [3]:
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
#Carrega e processo os dados para dentro de um RDD de LabeledPoint
data = MLUtils.loadLibSVMFile(sc, 'data/iris_libsvm.txt')

#Separa os dados em treino e teste
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [9]:
data.collect()[2]

LabeledPoint(0.0, (4,[0,1,2,3],[4.7,3.2,1.3,0.2]))

In [14]:
#Treina o modelo
#categoricalFeaturesInfo vazio indica que todas as features são contínuas
model = DecisionTree.trainClassifier(trainingData, numClasses=3, categoricalFeaturesInfo={}
                                    , impurity='gini', maxDepth=5, maxBins=32)

#Valida o modelo em instâncias de teste e calcula o erro no teste
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(
    lambda lp: lp[0] != lp[1]).count() / float(testData.count())
print('Test error: ' + str(testErr))
print('Learned classification tree model:')
print(model.toDebugString())

Test error: 0.041666666666666664
Learned classification tree model:
DecisionTreeModel classifier of depth 4 with 13 nodes
  If (feature 2 <= 2.45)
   Predict: 0.0
  Else (feature 2 > 2.45)
   If (feature 3 <= 1.75)
    If (feature 2 <= 5.05)
     Predict: 1.0
    Else (feature 2 > 5.05)
     If (feature 0 <= 6.05)
      Predict: 1.0
     Else (feature 0 > 6.05)
      Predict: 2.0
   Else (feature 3 > 1.75)
    If (feature 2 <= 4.85)
     If (feature 0 <= 5.95)
      Predict: 1.0
     Else (feature 0 > 5.95)
      Predict: 2.0
    Else (feature 2 > 4.85)
     Predict: 2.0



In [13]:
#Save and load the model
#model.save(sc, 'tmp/myDecisionTreeClassificationModel')
#sameModel = DecisionTreeModel.load(sc, 'tmp/myDecisionTreeClassificationModel')

Test error: 0.041666666666666664
Learned classification tree model:
DecisionTreeModel classifier of depth 4 with 13 nodes
  If (feature 2 <= 2.45)
   Predict: 0.0
  Else (feature 2 > 2.45)
   If (feature 3 <= 1.75)
    If (feature 2 <= 5.05)
     Predict: 1.0
    Else (feature 2 > 5.05)
     If (feature 0 <= 6.05)
      Predict: 1.0
     Else (feature 0 > 6.05)
      Predict: 2.0
   Else (feature 3 > 1.75)
    If (feature 2 <= 4.85)
     If (feature 0 <= 5.95)
      Predict: 1.0
     Else (feature 0 > 5.95)
      Predict: 2.0
    Else (feature 2 > 4.85)
     Predict: 2.0



---

### Hiperparâmetros
### Hiperparâmetros de especificação do problema
<b>numClasses</b>: Número de classes da variável dependente (Apenas para classificação) 

<b>categoricalFeaturesInfo</b>: Dicionário especificando quais variáveis independentes são categóricas, não é obrigatório mas pode ajudar no tempo de treinamento do modelo

### Critérios de parada do modelo
<b>maxDepth</b>: Profundidade máxima da árvore

<b>minInstancesPerNode</b>: Para que um ramo seja splitado, cada uma de suas folhas deve ter o mínimo de datapoints especificados neste parâmetro

<b>minInfoGain</b>: Para que um ramo seja splitado, deve ter um ganho de informação mínimo especificado neste parâmetro

### Parâmetros de tunagem
<b>maxBins</b>: Número máximo de bins utilizado para discretizar variáveis contínuas

<b>maxMemoryInMB</b>: Quantidade de memória que será utilizada para coletar estatísticas dos dados. Por padrão é 246mb para que o algoritmo rode na maioria dos ambientes. Aumentar este parâmetro pode deixar o treinamento mais rápido, se houver memória disponível

<b>subsamplingRate</b>: Fração dos dados de treinamento utilizados para treinar o modelo. Este parâmetro é relevante apenas para ensembles (RandomForest ou GradientBoostedTrees)

<b>impurity</b>: Especifica qual cálculo será utilizado para realizar o split 'gini' ou 'entropy'

---

### Árvore de decisão para classificação no pyspark.ML

Utilizando o exemplo do dataset da pesquisa de satisfação do aeroporto de São Francisco

In [10]:
from pyspark import SparkContext
from pyspark import sql

In [13]:
sc = SparkContext(master='local', appName='PesquisaSatisfacaoSFO')
spark = sql.SparkSession(sc)

In [22]:
pesquisa = spark.read.csv('data//2013_SFO_Customer_Survey.csv', inferSchema=True, header=True)

In [41]:
pesquisa.printSchema()

root
 |-- RESPNUM: integer (nullable = true)
 |-- CCGID: integer (nullable = true)
 |-- RUN: integer (nullable = true)
 |-- INTDATE: integer (nullable = true)
 |-- GATE: integer (nullable = true)
 |-- STRATA: integer (nullable = true)
 |-- PEAK: integer (nullable = true)
 |-- METHOD: integer (nullable = true)
 |-- AIRLINE: integer (nullable = true)
 |-- FLIGHT: integer (nullable = true)
 |-- DEST: integer (nullable = true)
 |-- DESTGEO: integer (nullable = true)
 |-- DESTMARK: integer (nullable = true)
 |-- ARRTIME: string (nullable = true)
 |-- DEPTIME: string (nullable = true)
 |-- Q2PURP1: integer (nullable = true)
 |-- Q2PURP2: integer (nullable = true)
 |-- Q2PURP3: integer (nullable = true)
 |-- Q2PURP4: integer (nullable = true)
 |-- Q2PURP5: integer (nullable = true)
 |-- Q2PURP6: string (nullable = true)
 |-- Q3GETTO1: integer (nullable = true)
 |-- Q3GETTO2: integer (nullable = true)
 |-- Q3GETTO3: integer (nullable = true)
 |-- Q3GETTO4: integer (nullable = true)
 |-- Q3GETT

In [56]:
colunasEstudo = pesquisa.columns[35:50]
df = pesquisa.select(colunasEstudo)

In [57]:
missingValuesInstrucao = map(lambda col: "'missingValues(" + col + ") " + col + "'", colunasEstudo)
", ".join(missingValuesInstrucao)

"'missingValues(Q7A_ART) Q7A_ART', 'missingValues(Q7B_FOOD) Q7B_FOOD', 'missingValues(Q7C_SHOPS) Q7C_SHOPS', 'missingValues(Q7D_SIGNS) Q7D_SIGNS', 'missingValues(Q7E_WALK) Q7E_WALK', 'missingValues(Q7F_SCREENS) Q7F_SCREENS', 'missingValues(Q7G_INFOARR) Q7G_INFOARR', 'missingValues(Q7H_INFODEP) Q7H_INFODEP', 'missingValues(Q7I_WIFI) Q7I_WIFI', 'missingValues(Q7J_ROAD) Q7J_ROAD', 'missingValues(Q7K_PARK) Q7K_PARK', 'missingValues(Q7L_AIRTRAIN) Q7L_AIRTRAIN', 'missingValues(Q7M_LTPARK) Q7M_LTPARK', 'missingValues(Q7N_RENTAL) Q7N_RENTAL', 'missingValues(Q7O_WHOLE) Q7O_WHOLE'"

In [59]:
df.selectExpr('avg(Q7O_WHOLE) Q7O_WHOLE').take(1)

[Row(Q7O_WHOLE=3.8743988684582744)]

In [73]:
medias = df.selectExpr('avg(Q7A_ART) Q7A_ART', 'avg(Q7B_FOOD) Q7B_FOOD', 'avg(Q7C_SHOPS) Q7C_SHOPS', 'avg(Q7D_SIGNS) Q7D_SIGNS', 'avg(Q7E_WALK) Q7E_WALK', 'avg(Q7F_SCREENS) Q7F_SCREENS', 'avg(Q7G_INFOARR) Q7G_INFOARR', 'avg(Q7H_INFODEP) Q7H_INFODEP', 'avg(Q7I_WIFI) Q7I_WIFI', 'avg(Q7J_ROAD) Q7J_ROAD', 'avg(Q7K_PARK) Q7K_PARK', 'avg(Q7L_AIRTRAIN) Q7L_AIRTRAIN', 'avg(Q7M_LTPARK) Q7M_LTPARK', 'avg(Q7N_RENTAL) Q7N_RENTAL')

In [80]:
medias.show()

+-----------------+-----------------+------------------+------------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+
|          Q7A_ART|         Q7B_FOOD|         Q7C_SHOPS|         Q7D_SIGNS|         Q7E_WALK|     Q7F_SCREENS|      Q7G_INFOARR|      Q7H_INFODEP|         Q7I_WIFI|         Q7J_ROAD|         Q7K_PARK|      Q7L_AIRTRAIN|       Q7M_LTPARK|       Q7N_RENTAL|
+-----------------+-----------------+------------------+------------------+-----------------+----------------+-----------------+-----------------+-----------------+-----------------+-----------------+------------------+-----------------+-----------------+
|4.300707213578501|3.951909476661952|3.9216407355021214|3.9315417256011314|4.114851485148515|4.02998585572843|4.645261669024046|4.667043847241867|4.271570014144271|4.501838755304102|4.825459688826025|4.8381895332390386|5.03281471004

In [81]:
treinamento = df.withColumn('label', df['Q7O_WHOLE']*1.0).na.replace(0,3).replace(6,3)

In [82]:
treinamento.show()

+-------+--------+---------+---------+--------+-----------+-----------+-----------+--------+--------+--------+------------+----------+----------+---------+-----+
|Q7A_ART|Q7B_FOOD|Q7C_SHOPS|Q7D_SIGNS|Q7E_WALK|Q7F_SCREENS|Q7G_INFOARR|Q7H_INFODEP|Q7I_WIFI|Q7J_ROAD|Q7K_PARK|Q7L_AIRTRAIN|Q7M_LTPARK|Q7N_RENTAL|Q7O_WHOLE|label|
+-------+--------+---------+---------+--------+-----------+-----------+-----------+--------+--------+--------+------------+----------+----------+---------+-----+
|      3|       4|        3|        3|       3|          3|          3|          3|       3|       3|       3|           3|         3|         3|        3|  3.0|
|      4|       4|        4|        4|       4|          3|          3|          3|       3|       4|       3|           3|         3|         3|        4|  4.0|
|      3|       4|        4|        2|       4|          4|          2|          2|       3|       2|       3|           4|         3|         4|        3|  3.0|
|      3|       3|        3|

In [88]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

colunasInput = colunasEstudo[0:-1]
va = VectorAssembler(inputCols=colunasInput, outputCol='features')
dtreg = DecisionTreeRegressor(labelCol='label', featuresCol='features', maxDepth=4)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='label')
grid = ParamGridBuilder().addGrid(dtreg.maxDepth, [3, 5, 7, 10]).build()
cv = CrossValidator(estimator=dtreg, estimatorParamMaps=grid, evaluator=evaluator, numFolds=10)
pipeline = Pipeline(stages=[va, dtreg])

In [94]:
modelo = pipeline.fit(treinamento)

In [99]:
display(modelo.stages[1])

DecisionTreeRegressionModel (uid=DecisionTreeRegressor_3118d32737cf) of depth 4 with 31 nodes

In [100]:
predicoes = modelo.transform(treinamento)

In [102]:
predicoes.show()

+-------+--------+---------+---------+--------+-----------+-----------+-----------+--------+--------+--------+------------+----------+----------+---------+-----+--------------------+------------------+
|Q7A_ART|Q7B_FOOD|Q7C_SHOPS|Q7D_SIGNS|Q7E_WALK|Q7F_SCREENS|Q7G_INFOARR|Q7H_INFODEP|Q7I_WIFI|Q7J_ROAD|Q7K_PARK|Q7L_AIRTRAIN|Q7M_LTPARK|Q7N_RENTAL|Q7O_WHOLE|label|            features|        prediction|
+-------+--------+---------+---------+--------+-----------+-----------+-----------+--------+--------+--------+------------+----------+----------+---------+-----+--------------------+------------------+
|      3|       4|        3|        3|       3|          3|          3|          3|       3|       3|       3|           3|         3|         3|        3|  3.0|[3.0,4.0,3.0,3.0,...| 3.676829268292683|
|      4|       4|        4|        4|       4|          3|          3|          3|       3|       4|       3|           3|         3|         3|        4|  4.0|[4.0,4.0,4.0,4.0,...| 3.6768292

In [103]:
avaliador = RegressionEvaluator()
avaliador.evaluate(predicoes, {evaluator.metricName: 'rmse'})

0.555808023551782

### feature importance

In [105]:
modelo.stages[1].featureImportances

SparseVector(14, {0: 0.0653, 1: 0.1173, 2: 0.0099, 3: 0.5219, 4: 0.0052, 5: 0.2403, 8: 0.0028, 10: 0.0059, 13: 0.0314})

In [107]:
importanciaFeatures = modelo.stages[1].featureImportances.toArray()

In [109]:
nomeFeatures = map(lambda feat: feat.name, df.schema.fields)

In [111]:
importanciaFeaturesMap = zip(importanciaFeatures, nomeFeatures)

In [113]:
dfImportanciaFeatures = spark.createDataFrame(sc.parallelize(importanciaFeaturesMap).map(lambda r: [r[1], float(r[0])]))

In [115]:
dfImportanciaFeatures.show()

+------------+--------------------+
|          _1|                  _2|
+------------+--------------------+
|     Q7A_ART| 0.06525145453249198|
|    Q7B_FOOD| 0.11726445589519406|
|   Q7C_SHOPS|0.009940295196927703|
|   Q7D_SIGNS|  0.5219302904331116|
|    Q7E_WALK|0.005232136454289954|
| Q7F_SCREENS|   0.240344587239121|
| Q7G_INFOARR|                 0.0|
| Q7H_INFODEP|                 0.0|
|    Q7I_WIFI|0.002774900310175427|
|    Q7J_ROAD|                 0.0|
|    Q7K_PARK|0.005854457703262646|
|Q7L_AIRTRAIN|                 0.0|
|  Q7M_LTPARK|                 0.0|
|  Q7N_RENTAL| 0.03140742223542566|
+------------+--------------------+



In [116]:
dfImportanciaFeatures = dfImportanciaFeatures.withColumnRenamed('_1', 'Feature').withColumnRenamed('_2', 'Importancia')

In [122]:
dfImportanciaFeatures.orderBy('Importancia').show()

+------------+--------------------+
|     Feature|         Importancia|
+------------+--------------------+
|Q7L_AIRTRAIN|                 0.0|
| Q7H_INFODEP|                 0.0|
|  Q7M_LTPARK|                 0.0|
|    Q7J_ROAD|                 0.0|
| Q7G_INFOARR|                 0.0|
|    Q7I_WIFI|0.002774900310175427|
|    Q7E_WALK|0.005232136454289954|
|    Q7K_PARK|0.005854457703262646|
|   Q7C_SHOPS|0.009940295196927703|
|  Q7N_RENTAL| 0.03140742223542566|
|     Q7A_ART| 0.06525145453249198|
|    Q7B_FOOD| 0.11726445589519406|
| Q7F_SCREENS|   0.240344587239121|
|   Q7D_SIGNS|  0.5219302904331116|
+------------+--------------------+

